In [84]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel.data import PanelData
from linearmodels.panel import PanelOLS, PooledOLS, RandomEffects, compare
import matplotlib.pyplot as plt
import pyblp

In [85]:
data=pd.read_csv("../data/merged/len4_ndb_agg_blp_DropNever.csv",encoding="shift-jis",index_col=0)
data["market_ids"]=data["id"].astype(int).astype(str)+"-"+data["year"].astype(str)
data.rename(columns={"薬価":"prices","r_share":"shares",
                    "elasped_0":"demand_instruments0",
                    "elasped_1":"demand_instruments1",
                    'elasped_2': 'demand_instruments2',
                    'elasped_3': 'demand_instruments3',
                    'elasped_4': 'demand_instruments4',
                    'elasped_5': 'demand_instruments5',
                    'elasped_6': 'demand_instruments6',
                    'elasped_7': 'demand_instruments7',
                    'elasped_8': 'demand_instruments8',
                    'elasped_9': 'demand_instruments9'},inplace=True)
data=data.loc[data["shares"]>0]
# data["shares"]=data["r_share"]
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,prices,後発品区分,総計,year,in_hospital,...,id_3992.0,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0,q_share,shares,market_ids
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,0,0,0,0,0,0,0,0.182126,0.182126,1145-2014
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,0,0,0,0,0,0,0,0.080302,0.080302,1145-2014
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,0,0,0,0,0,0,0,0.066275,0.066275,1145-2014
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,0,0,0,0,0,0,0,0.410299,0.410299,1145-2014
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,0,0,0,0,0,0,0,0.260997,0.260997,1145-2014


In [86]:
logit_formulation = pyblp.Formulation('prices+oral+in_hospital+後発品区分', absorb='C(id)+C(year)')
logit_formulation

prices + oral + in_hospital + 後発品区分 + Absorb[C(id)] + Absorb[C(year)]

In [87]:
data["shares"]-=1e-10
problem = pyblp.Problem(logit_formulation, data)
problem

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:01.

Dimensions:
 T     N      K1    MD    ED 
---  ------  ----  ----  ----
344  104614   4     13    2  

Formulations:
     Column Indices:          0      1         2         3  
--------------------------  ------  ----  -----------  -----
X1: Linear Characteristics  prices  oral  in_hospital  後発品区分


Dimensions:
 T     N      K1    MD    ED 
---  ------  ----  ----  ----
344  104614   4     13    2  

Formulations:
     Column Indices:          0      1         2         3  
--------------------------  ------  ----  -----------  -----
X1: Linear Characteristics  prices  oral  in_hospital  後発品区分

In [88]:
logit_results = problem.solve()
logit_results

Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 1    +3.265288E+02     0      +7.333794E+00  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +8.577849E+01     0      +9.279003E+00      +7.183195E+06  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices            oral          in_hospital         後発品区分     
---------------  ---------------  ---------------  ---------------
 -8.963819E-04    +2.241770E+00    +2

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +8.577849E+01     0      +9.279003E+00      +7.183195E+06  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices            oral          in_hospital         後発品区分     
---------------  ---------------  ---------------  ---------------
 -8.963819E-04    +2.241770E+00    +2.547620E-01    -1.835916E+00 
(+4.575067E-04)  (+5.899900E-02)  (+1.513933E-02)  (+1.395358E-01)